In [36]:
import os
import cv2
import pydicom
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

%matplotlib notebook

In [38]:
def loadFile(filename):
    dicom = pydicom.read_file(filename)
    vid = dicom.pixel_array
    return vid

# CURRENTLY NOT IN USE
def preProcess(image_array, clip_limit = 3):
    img_array_list = []
    for image in image_array:
        # convert image to LAB color model
        image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

        # split the image into L, A, and B channels
        l_channel, a_channel, b_channel = cv2.split(image_lab)

        # apply CLAHE to lightness channel
        clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(4, 4))
        cl = clahe.apply(l_channel)

        # merge the CLAHE enhanced L channel with the original A and B channel
        merged_channels = cv2.merge((cl, a_channel, b_channel))

        # convert iamge from LAB color model back to RGB color model
        img = cv2.cvtColor(merged_channels, cv2.COLOR_LAB2BGR)
        final_image = cv2.bilateralFilter(img, 7, sigmaSpace = 75, sigmaColor =75)
        img_array_list.append(final_image)
    
    image_array_limited_equalized = np.array(img_array_list)
    return image_array_limited_equalized


def writeVideo(img_array, filename, directory):
    frame_num, height, width, ch = img_array.shape
    filename_output = directory + '/' + filename + '.avi'        
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video = cv2.VideoWriter(filename_output, fourcc, 15, (width, height))       
    for img in img_array:
#         img_rgb = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        video.write(img) # Write video file frame by frame
    video.release()

In [ ]:
writeVideo(vid1.pixel_array, output_directory)

### Convert dicoms in each class to AVI


In [42]:
classes = {'apex', 'mitral', 'papillary'}
for clas in classes:
    parent_dir = os.path.dirname(os.getcwd())
    avi_dir = os.path.join(parent_dir, "data", clas, "avi")
    dicom_dir = os.path.join(parent_dir, "Data", clas, "dicom")
    
    video_list = os.listdir(dicom_dir)
    if '.DS_Store' in video_list: video_list.remove('.DS_Store')
    
    for video_name in tqdm(video_list):
        vid = loadFile(os.path.join(dicom_dir, video_name))
        writeVideo(vid, video_name[:-4], avi_dir)